In [1]:
import edge_tts
import asyncio
import util    
import json
import sys
from IPython.display import Audio
from speakers.processors import load_preprocess,EdgeProcessorData,get_processors
from speakers.common.utils import get_abs_path
from omegaconf import OmegaConf


2023-08-26 21:52:44 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


from speakers.common.registry import registry rvc_speakers


/media/gpt4-pdf-chatbot-langchain/pyenv-rvc-speakers/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


from speakers.common.registry import registry vits_to_voice
from speakers.common.registry import registry bark_to_voice
from speakers.common.registry import registry edge_to_voice
register_bootstrap runner_bootstrap_web
from speakers.common.registry import registry vits_voice_task
from speakers.common.registry import registry bark_voice_task
 Key with which the item will be registered {'device': 'cuda:0'}
 Key with which the item will be registered {'device': 'cuda:0', 'is_half': True}
 Key with which the item will be registered {'device': 'cuda:0', 'is_half': True, 'x_pad': 3}
 Key with which the item will be registered {'device': 'cuda:0', 'is_half': True, 'x_pad': 3, 'x_query': 10}
 Key with which the item will be registered {'device': 'cuda:0', 'is_half': True, 'x_pad': 3, 'x_query': 10, 'x_center': 60}
 Key with which the item will be registered {'device': 'cuda:0', 'is_half': True, 'x_pad': 3, 'x_query': 10, 'x_center': 60, 'x_max': 65}


In [3]:
import logging
from speakers.processors.edge_to_voice import set_edge_to_voice_logger 
from speakers.processors.rvc_speakers_processor import set_rvc_speakers_logger
from speakers.processors.vits_to_voice import set_vits_to_voice_logger
from speakers.server.servlet.runner import set_server_runner_logger
from speakers.common.log import get_logger, set_log_level
set_log_level(level=logging.DEBUG) 
logger = get_logger('notebook')
set_rvc_speakers_logger(logger)
set_vits_to_voice_logger(logger)
set_edge_to_voice_logger(logger)
set_server_runner_logger(logger)

In [4]:
 

speakers_config_file='speakers.yaml'
config = OmegaConf.load(get_abs_path(speakers_config_file))
print(config)
load_preprocess(config=config.get('preprocess'))
 
 

[notebook] limit text and audio length in huggingface spaces: False
[notebook] _load_voice_mode: /media/gpt4-pdf-chatbot-langchain/RVC-Speakers/vits/model/config.json


{'preprocess': [{'vits_processor': {'name': 'vits_to_voice', 'vits_model_path': 'model/G_953000.pth', 'voice_config_file': 'model/config.json'}}, {'rvc_processor': {'name': 'rvc_speakers', 'hubert_model_path': 'model/hubert_base.pt', 'rvc_config_file': 'rvc.yaml'}}, {'edge_processor': {'name': 'edge_to_voice'}}], 'tasks': [{'vits_voice_task': {'name': 'vits_voice_task', 'preprocess': [{'vits': {'processor': 'vits_processor', 'processor_name': 'VITS'}}, {'rvc': {'processor': 'rvc_processor', 'processor_name': 'RVC'}}]}}, {'bark_voice_task': {'name': 'bark_voice_task', 'preprocess': [{'bark': {'processor': 'bark_processor', 'processor_name': 'BARK'}}, {'rvc': {'processor': 'rvc_processor', 'processor_name': 'RVC'}}]}}], 'bootstrap': [{'runner_bootstrap_web': {'name': 'runner_bootstrap_web', 'host': '0.0.0.0', 'port': 10001}}]}
{'name': 'vits_to_voice', 'vits_model_path': 'model/G_953000.pth', 'voice_config_file': 'model/config.json'}


[notebook] Models loaded vits_to_voice
[notebook] Load hubert model/media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/model/hubert_base.pt


{'name': 'rvc_speakers', 'hubert_model_path': 'model/hubert_base.pt', 'rvc_config_file': 'rvc.yaml'}


[notebook] Loaded hubert model
[notebook] Models Load:rvc_speakers
[notebook] rmvpe_path:/media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/model/rmvpe.pt
[notebook] Loading model: yiqing
[notebook] Loading model model_info_config_file: /media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/model/yiqing/config.json
[notebook] <All keys matched successfully>
[notebook] Loading model: bob
[notebook] Loading model model_info_config_file: /media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/model/bob/config.json


...

gin_channels: 256 self.spk_embed_dim: 109


[notebook] <All keys matched successfully>
[notebook] Loading model: maomao
[notebook] Loading model model_info_config_file: /media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/model/maomao/config.json


gin_channels: 256 self.spk_embed_dim: 109


[notebook] <All keys matched successfully>
[notebook] Models loaded:rvc_speakers, len:6


gin_channels: 256 self.spk_embed_dim: 109
{'name': 'edge_to_voice'}


In [5]:

edge_to_voice = get_processors('edge_processor')

In [6]:

tts_speaker = [
    '%s (%s)' % (
        s['FriendlyName'],
        s['Gender'], 
    )
    for s in edge_to_voice.tts_speakers_list
] 

tts_speaker_index = tts_speaker.index('Microsoft Xiaoxiao Online (Natural) - Chinese (Mainland) (Female)')
tts_speaker_index

55

In [7]:
rate='+20%'
volume='-50%' 

data=EdgeProcessorData(text="我是你的情感导师，青春的时间转瞬即逝，秋水于海又岂有一只浮游",
                      tts_speaker=tts_speaker_index,
                      rate=rate,
                      volume=volume)
data

EdgeProcessorData(text='我是你的情感导师，青春的时间转瞬即逝，秋水于海又岂有一只浮游', tts_speaker=55, rate='+20%', volume='-50%')

In [8]:
tts_np, tts_sr = edge_to_voice(data=data)

In [9]:
Audio(tts_np, rate=tts_sr)